### TODO
- Now we can take the special cases we added to info_grupos back to frac_vars.xlsx since we have them in cols to avoid.
- The HelperFunctions can raise Errors or warning that help us identify null values, mismatching vars and things like that.

In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
import pickle
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
from info_grupos import empirical_vars_to_avoid, frac_vars_special_cases_list
from genera_muestra import GenerateLHS
from utils import HelperFunctions

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf
import sisepuede as si

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
target_country = 'croatia'
experiment_id = 1


In [4]:

FILE_PATH = os.getcwd()
build_path = lambda PATH : os.path.abspath(os.path.join(*PATH))

DATA_PATH = build_path([FILE_PATH, "..", "data"])
OUTPUT_PATH = build_path([FILE_PATH, "..", "output"])

SSP_OUTPUT_PATH = build_path([OUTPUT_PATH, "ssp"])

REAL_DATA_FILE_PATH = build_path([DATA_PATH, "real_data.csv"]) 

SALIDAS_EXPERIMENTOS_PATH = build_path([OUTPUT_PATH, "experiments"]) 

INPUTS_ESTRESADOS_PATH = build_path([SALIDAS_EXPERIMENTOS_PATH, "sim_inputs"])
OUTPUTS_ESTRESADOS_PATH = build_path([SALIDAS_EXPERIMENTOS_PATH, "sim_outputs"])
helper_functions = HelperFunctions()

helper_functions.ensure_directory_exists(INPUTS_ESTRESADOS_PATH)
helper_functions.ensure_directory_exists(OUTPUTS_ESTRESADOS_PATH)

Directory already exists: /home/tony-ubuntu/decision_sciences/lhs_ssp_sampling/output/experiments/sim_inputs
Directory already exists: /home/tony-ubuntu/decision_sciences/lhs_ssp_sampling/output/experiments/sim_outputs


In [5]:
### Cargamos datos de ejemplo de costa rica

examples = SISEPUEDEExamples()
cr = examples("input_data_frame")

In [6]:
df_input = pd.read_csv(REAL_DATA_FILE_PATH)
df_input.head()

,region,iso_code3,period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,croatia,HRV,0,8807000,70,2923,31.751466,6468,315,508,...,5.546667,28.8742,0.602367,1.930675,0.5205,2.638183,0,0,35.76480,21.067738
1,croatia,HRV,1,8807000,70,2923,31.751466,6468,315,508,...,5.555383,29.6558,0.479900,2.304050,0.7342,3.256933,0,0,47.57660,20.412554
2,croatia,HRV,2,8807000,70,2923,31.751466,6468,315,508,...,4.304906,30.0390,0.360400,2.075950,0.8615,3.199083,0,0,41.09780,22.329531
3,croatia,HRV,3,8807000,70,2923,31.751466,6468,315,508,...,6.272229,30.0390,0.160300,2.662133,0.9852,3.266680,0,0,37.42445,21.505800
4,croatia,HRV,4,8807000,70,2923,31.751466,6468,315,508,...,5.878853,30.0390,0.196325,2.543567,0.9045,4.068040,0,0,39.81490,21.874247


In [7]:
df_input = df_input.rename(columns={'period':'time_period'})
df_input = helper_functions.add_missing_cols(cr, df_input.copy())
df_input = df_input.drop(columns='iso_code3')

In [8]:
df_input.head()

,region,time_period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,croatia,0,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,croatia,1,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,croatia,2,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,croatia,3,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,croatia,4,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [9]:
# Double checking that our df is in the correct shape MAKE SURE THIS IS OK THEY HAVE TO BE EQUAL!
helper_functions.compare_dfs(cr, df_input)

Columns in df1 but not in df2: set()
Columns in df2 but not in df1: set()
Columns shared in both df1 and df2: {'frac_inen_energy_metals_coal', 'scalar_inen_energy_demand_rubber_and_leather', 'nemomod_entc_renewable_energy_tag_st_flywheels', 'frac_agrc_production_lost_sent_to_msw', 'frac_wali_ww_domestic_urban_treatment_path_advanced_aerobic', 'nemomod_entc_emissions_activity_ratio_scalar_pp_biogas_n2o', 'nemomod_entc_total_annual_min_capacity_investment_pp_ocean_gw', 'efficfactor_scoe_heat_energy_commercial_municipal_coal', 'demscalar_ippu_product_use_other', 'consumpinit_inen_energy_tj_per_tonne_production_electronics', 'nemomod_entc_residual_capacity_pp_geothermal_gw', 'frac_lndu_grasslands_cl2_wet', 'elasticity_lvst_sheep_demand_to_gdppc', 'efficfactor_enfu_industrial_energy_fuel_oil', 'nemomod_entc_total_annual_min_capacity_investment_pp_hydropower_gw', 'nemomod_entc_variable_cost_pp_waste_incineration_usd_per_mwh', 'ef_enfu_combustion_tonne_co2_per_tj_fuel_hydrogen', 'exports_lvst

In [10]:
# Checking if there are any columns with null values in it
helper_functions.get_cols_with_nans(df_input)

[]


[]

In [11]:
columns_all_999 = df_input.columns[(df_input == -999).any()].tolist()
columns_all_999

['frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biogas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biomass',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_geothermal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_hydropower',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_nuclear',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_ocean',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_oil',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_solar',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_waste_incineration',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_wind',
 'limit_gnrl_annual_emissions_mt_ch

In [12]:
empirical_vars_to_avoid

['area_gnrl_country_ha',
 'gdp_mmm_usd',
 'occrateinit_gnrl_occupancy',
 'population_gnrl_rural',
 'population_gnrl_urban',
 'lndu_reallocation_factor']

In [13]:
df_input['lndu_reallocation_factor'].head()

0    0
1    0
2    0
3    0
4    0
Name: lndu_reallocation_factor, dtype: int64

In [14]:
# Avoid land use stuff and some frac special cases
pij_cols = [col for col in df_input.columns if col.startswith('pij')]
cols_to_avoid = pij_cols + frac_vars_special_cases_list + columns_all_999 + empirical_vars_to_avoid
cols_to_stress = helper_functions.get_indicators_col_names(df_input, cols_with_issue=cols_to_avoid)

In [15]:
df_input['ef_lndu_conv_croplands_to_forests_secondary_gg_co2_ha'].head()

0    0
1    0
2    0
3    0
4    0
Name: ef_lndu_conv_croplands_to_forests_secondary_gg_co2_ha, dtype: int64

In [16]:
# # Add a small epsilon to variables that are full of zeros but not frac_ columns
# epsilon = 1e-6
# non_frac_cols_to_stress = [col for col in cols_to_stress if not col.startswith('frac_')]
# for col in non_frac_cols_to_stress:
#     if (df_input[col] == 0).all():  # Check if the column is full of zeros
#         df_input[col] += epsilon  # Add epsilon to the entire column

In [17]:
df_input['ef_lndu_conv_croplands_to_forests_secondary_gg_co2_ha'].head()

0    0
1    0
2    0
3    0
4    0
Name: ef_lndu_conv_croplands_to_forests_secondary_gg_co2_ha, dtype: int64

In [18]:
# Defines upper bound to pass to GenerateLHS
u_bound = 2

# Defines number of sample vectors that GenerateLHS will create
n_arrays = 100
sampling_file_path = os.path.join('sampling_files', f'sample_scaled_{n_arrays}_{u_bound}.pickle') 

# Generates sampling matrix
if not os.path.exists(sampling_file_path):
    # Generates sampling matrix if it does not exist
    generate_sample = GenerateLHS(n_arrays, n_var=len(cols_to_stress), u_bound=u_bound)
    generate_sample.generate_sample()

# Load the sampling matrix
with open(sampling_file_path, 'rb') as handle:
    sample_scaled = pickle.load(handle)

In [19]:
lhs_sampling_array = sample_scaled[experiment_id]

In [20]:
print(lhs_sampling_array)
print(lhs_sampling_array.shape)

[0.68131578 0.56667986 1.26215516 ... 1.01133995 0.57521847 1.51992531]
(2033,)


In [21]:
# # Plotting the histogram
# plt.figure(figsize=(8, 5))
# plt.hist(sample_scaled[1], bins=30, edgecolor='black', alpha=0.7)
# plt.title('Histogram of Sampled Random Values')
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.grid(axis='y', alpha=0.75)
# plt.show()

In [22]:
# Creating new df with the sampled data
stressed_df = df_input.copy()
stressed_df[cols_to_stress] = (df_input[cols_to_stress] * sample_scaled[experiment_id]).to_numpy()

In [23]:
stressed_df['lndu_reallocation_factor'].head()

0    0
1    0
2    0
3    0
4    0
Name: lndu_reallocation_factor, dtype: int64

In [24]:
stressed_df['ef_lndu_conv_croplands_to_forests_secondary_gg_co2_ha'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: ef_lndu_conv_croplands_to_forests_secondary_gg_co2_ha, dtype: float64

In [25]:
stressed_df['frac_agrc_bevs_and_spices_cl1_temperate'].head()

0    0.722487
1    0.722487
2    0.722487
3    0.722487
4    0.722487
Name: frac_agrc_bevs_and_spices_cl1_temperate, dtype: float64

In [26]:
stressed_df['frac_agrc_bevs_and_spices_cl1_tropical'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: frac_agrc_bevs_and_spices_cl1_tropical, dtype: float64

In [27]:
# Load new groups that need normalization
df_frac_vars = pd.read_excel('frac_vars.xlsx', sheet_name='frac_vars_no_special_cases')
df_frac_vars.head()

,frac_var_name,frac_var_name_prefix
0,frac_agrc_bevs_and_spices_cl1_temperate,frac_agrc_bevs_and_spices_cl1
1,frac_agrc_bevs_and_spices_cl1_tropical,frac_agrc_bevs_and_spices_cl1
2,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_bevs_and_spices_cl2
3,frac_agrc_bevs_and_spices_cl2_wet,frac_agrc_bevs_and_spices_cl2
4,frac_agrc_cereals_cl1_temperate,frac_agrc_cereals_cl1


In [28]:
need_norm_prefix = df_frac_vars.frac_var_name_prefix.unique()

In [29]:
# Normalizing frac_ var groups using softmax
df_frac_vars = pd.read_excel('frac_vars.xlsx', sheet_name='frac_vars_no_special_cases')
need_norm_prefix = df_frac_vars.frac_var_name_prefix.unique()

for subgroup in need_norm_prefix:
    subgroup_cols = [i for i in stressed_df.columns if subgroup in i]
    
    # Skip normalization for columns in cols_to_avoid
    if any(col in cols_to_avoid for col in subgroup_cols):
        continue

    # Apply softmax normalization
    stressed_df[subgroup_cols] = stressed_df[subgroup_cols].apply(
        lambda row: np.exp(row) / np.exp(row).sum(), axis=1
    )

# Special case for ce_problematic
ce_problematic = [
    'frac_waso_biogas_food',
    'frac_waso_biogas_sludge',
    'frac_waso_biogas_yard',
    'frac_waso_compost_food',
    'frac_waso_compost_methane_flared',
    'frac_waso_compost_sludge',
    'frac_waso_compost_yard'
]

# Apply softmax normalization
stressed_df[ce_problematic] = stressed_df[ce_problematic].apply(
    lambda row: np.exp(row) / np.exp(row).sum(), axis=1
)

In [30]:
print(stressed_df['frac_agrc_bevs_and_spices_cl1_temperate'].head())
print(stressed_df['frac_agrc_bevs_and_spices_cl1_tropical'].head())

0    0.673154
1    0.673154
2    0.673154
3    0.673154
4    0.673154
Name: frac_agrc_bevs_and_spices_cl1_temperate, dtype: float64
0    0.326846
1    0.326846
2    0.326846
3    0.326846
4    0.326846
Name: frac_agrc_bevs_and_spices_cl1_tropical, dtype: float64


In [31]:
stressed_df['frac_agrc_bevs_and_spices_cl1_tropical'].head() + stressed_df['frac_agrc_bevs_and_spices_cl1_temperate'].head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

####################

In [32]:
# # Assuming df_estresado is defined and contains columns
# vars_grupo = [i for i in df_estresado.columns if i.startswith('frac_')]

# df_frac_vars = pd.DataFrame(vars_grupo, columns=['frac_var_name'])
# df_frac_vars.sort_values(by='frac_var_name', inplace=True)

# # Extract prefix by removing the last '_{word}' segment
# df_frac_vars['frac_var_name_prefix'] = df_frac_vars['frac_var_name'].apply(lambda x: '_'.join(x.split('_')[:-1]))

# df_frac_vars.to_csv('frac_vars.csv', index=False)

###########

In [33]:
# Checking if there are any columns with null values in it
columns_with_na = helper_functions.get_cols_with_nans(stressed_df)
if columns_with_na:
    stressed_df[columns_with_na] = stressed_df[columns_with_na].fillna(0)

helper_functions.get_cols_with_nans(stressed_df)

[]
[]


[]

In [34]:
transformers = trf.transformers.Transformers(
    {},
    df_input = stressed_df,
)

##  SETUP SOME SISEPUEDE STUFF

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

# set an ouput path and instantiate

trf.instantiate_default_strategy_directory(
        transformers,
        SSP_OUTPUT_PATH,
    )

# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
        SSP_OUTPUT_PATH,
        transformers = transformers,
    )

strategies = trf.Strategies(
        transformations,
        export_path = "transformations",
        prebuild = True,
    )

In [35]:


# call the example
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
        df_trajgroup = df_vargroups,
        include_simplex_group_as_trajgroup = True,
        strategies = [0, 1000],
    )



0

In [36]:
import sisepuede as si
ssp = si.SISEPUEDE(
        "calibrated",
        initialize_as_dummy = False, # no connection to Julia is initialized if set to True
        regions = [target_country],
        db_type = "csv",
        strategies = strategies,
        try_exogenous_xl_types_in_variable_specification = True,
    )

2024-11-26 10:18:11,158 - INFO - Successfully initialized SISEPUEDEFileStructure.
2024-11-26 10:18:11,161 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2024-11-26 10:18:11,161 - INFO - 	Setting export engine to 'csv'.
2024-11-26 10:18:11,162 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2024-11-26 10:18:11,162 - INFO - Successfully instantiated table ANALYSIS_METADATA
2024-11-26 10:18:11,163 - WARNING - No index fields found in ATTRIBUTE_DESIGN. Initializing index fields.
2024-11-26 10:18:11,164 - INFO - Successfully instantiated table ATTRIBUTE_DESIGN
2024-11-26 10:18:11,165 - WARNING - No index fields found in ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES. Initializing index fields.
2024-11-26 10:18:11,166 - INFO - Successfully instantiated table ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES
2024-11-26 10:18:11,167 - WARNING - No index fi

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   2470.8 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
2024-11-26 10:19:09,651 - INFO - Successfully initialized JuMP optimizer from solver module HiGHS.
2024-11-26 10:19:09,675 - INFO - Successfully initialized SISEPUEDEModels.
2024-11-26 10:19:09,683 - INFO - Table ANALYSIS_METADATA successfully written to /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11

In [37]:
# Checks if the land use reallocation factor is set to 0.0
helper_functions.check_land_use_factor(ssp_object=ssp, target_country=target_country)

In [38]:
# Create parameters dict for the model to run
dict_run = {
        ssp.key_future: [0],
        ssp.key_design: [0],
        ssp.key_strategy: [
            0,
            1000,
        ],
    }


In [39]:
# we'll save inputs since we're doing a small set of runs
ssp.project_scenarios(
        dict_run,
        save_inputs = True,
    )

2024-11-26 10:19:13,786 - INFO - 
***	STARTING REGION croatia	***



2024-11-26 10:19:17,650 - INFO - Trying run primary_id = 0 in region croatia
2024-11-26 10:19:17,651 - INFO - Running AFOLU model
2024-11-26 10:19:17,870 - INFO - AFOLU model run successfully completed
2024-11-26 10:19:17,871 - INFO - Running CircularEconomy model
2024-11-26 10:19:17,929 - INFO - CircularEconomy model run successfully completed
2024-11-26 10:19:17,930 - INFO - Running IPPU model
2024-11-26 10:19:18,023 - INFO - IPPU model run successfully completed
2024-11-26 10:19:18,024 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2024-11-26 10:19:18,044 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2024-11-26 10:19:18,151 - INFO - EnergyConsumption without Fugitive Emissions model run successfully completed
2024-11-26 10:19:18,152 - INFO - Running Energy model (Electricity and Fuel Production: trying to call Julia)
2024-11-26 10:19:18,189 - INFO - 	Path to temporary

2024-26-Nov 10:19:18.934 Opened SQLite database at /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-26-Nov 10:19:19.290 Added NEMO structure to SQLite database at /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-26-Nov 10:19:36.450 Started modeling scenario. NEMO version = 2.0.0, solver = HiGHS.
2024-26-Nov 10:20:46.294 Finished modeling scenario.


┌ Warning: Model period activity upper limits for technologies (TotalTechnologyModelPeriodActivityUpperLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:479
┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:6112
2024-11-26 10:20:46,633 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-26 10:20:46,651 - INFO - EnergyProduction model run successfully completed
2024-11-26 10:20:46,652 - INFO - Running Energy (Fugitive Emissions)
2024-11-26 10:20:46,703 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-26 10:20:46,704 - INFO - Appending Socioeconomic outputs
2024-11-26 10:20:46,715 - INFO - Socioeconomic outputs successfully appended.
2024-11-26 10:20:46,720 - INFO - Model run for primary_id = 0 successfu

2024-26-Nov 10:20:49.170 Started modeling scenario. NEMO version = 2.0.0, solver = HiGHS.


┌ Warning: Model period activity upper limits for technologies (TotalTechnologyModelPeriodActivityUpperLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:479
┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:6112


2024-26-Nov 10:21:34.602 Finished modeling scenario.


2024-11-26 10:21:34,820 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-26 10:21:34,835 - INFO - EnergyProduction model run successfully completed
2024-11-26 10:21:34,836 - INFO - Running Energy (Fugitive Emissions)
2024-11-26 10:21:34,887 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-26 10:21:34,888 - INFO - Appending Socioeconomic outputs
2024-11-26 10:21:34,898 - INFO - Socioeconomic outputs successfully appended.
2024-11-26 10:21:34,902 - INFO - Model run for primary_id = 1001 successfully completed in 48.17 seconds (n_tries = 1).
2024-11-26 10:21:34,922 - INFO - 
***	 REGION croatia COMPLETE	***

2024-11-26 10:21:35,008 - INFO - Table MODEL_OUTPUT successfully written to /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/out/sisepuede_run_2024-11-26T10;18;10.536708/sisepuede_run_2024-11-26T10;18;10.536708_output_database/MODEL_OUTPUT.csv.
2024-11-26 10:21:35,012 - INFO - Table ATTRIBUTE_PR

{'croatia': [0, 1001]}

In [40]:
INPUTS_ESTRESADOS_FILE_PATH = build_path([INPUTS_ESTRESADOS_PATH, f"sim_input_{experiment_id}.csv"])
OUTPUTS_ESTRESADOS_FILE_PATH = build_path([OUTPUTS_ESTRESADOS_PATH, f"sim_output_{experiment_id}.csv"])


df_out = ssp.read_output(None)
df_out.to_csv(OUTPUTS_ESTRESADOS_FILE_PATH, index=False)
stressed_df.to_csv(INPUTS_ESTRESADOS_FILE_PATH, index=False)